## **Import Packages**

In [1]:
import pandas as pd
import itertools
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tqdm

## **Load Data and Remove Unneccessary Columns**

In [2]:
data = pd.read_excel('Stock-Data-Optimization-2-Percent-06-29-2021.xlsm', sheet_name = 'Data')
data = data.drop(data.columns[[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 48]], axis = 1) #drop the calculated column

## **Make the First Row as Column Header**

In [3]:
new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

## **User Input for Static - Method 2**

In [4]:
lower_bound_sma20 = -5
upper_bound_sma20 = -2
momentum = 0
up_from_day = 0
up_from_open = 9
lower_bound_rsi = 64
upper_bound_rsi = 70
volume_ratio = 1.5
market_cap = 2000

min_event = 30

## **Static - Method 2**

In [5]:
data['mrkt cap'] = data['mrkt cap'].fillna(0) #Fill Blank space in market cap with 0 
data['Volume Ratio'] = data['Volume Ratio'].fillna(0) #Fill Blank space in volume ratio with 0

#Creating columns AH to AP
data.loc[(data['20SMA'] > data['50SMA'] ) , 'sma20 > sma50'] = 0
data['sma20 > sma50'] = data['sma20 > sma50'].fillna(1)
data.loc[(data['20SMA %'] < upper_bound_sma20) & (data['20SMA %'] > lower_bound_sma20) , 'sma20 bound check'] = 0
data['sma20 bound check'] = data['sma20 bound check'].fillna(1)
data.loc[(data['Mom'] > momentum) , 'momentum'] = 0
data['momentum'] = data['momentum'].fillna(1)
data.loc[(data['%Up'] > up_from_day) , '8% Daily'] = 0
data['8% Daily'] = data['8% Daily'].fillna(1)
data.loc[(data['% Open'] > up_from_open) , '2% UP'] = 0
data['2% UP'] = data['2% UP'].fillna(1)
data.loc[(data['14minRSI'] < upper_bound_rsi) & (data['14minRSI'] > lower_bound_rsi) , 'RSI (60-72)'] = 0
data['RSI (60-72)'] = data['RSI (60-72)'].fillna(1)
data.loc[(data['Volume Ratio'] < volume_ratio) , 'Volume Ration'] = 0
data['Volume Ration'] = data['Volume Ration'].fillna(1)
data.loc[(data['mrkt cap'] < market_cap) , 'market cap'] = 1
data['market cap'] = data['market cap'].fillna(0)
data.loc[(data['Macd'] == 'Pass') , 'MACD'] = 0
data['MACD'] = data['MACD'].fillna(1)

#Creating all the On, Off scenarios
switch = list(itertools.product(['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF'],\
                       ['ON', 'OFF'], ['ON', 'OFF'], ['ON', 'OFF']))

df = data[['sma20 > sma50','sma20 bound check', 'momentum', '8% Daily', '2% UP', 'RSI (60-72)', 'Volume Ration', \
           'market cap', 'MACD', 'win', 'upwin', 'Loss', 'update loss' ]] #Selecting relevant columns

#Creating empty lists and Dataframe to be used in iterations
indices = []
col_name = []
result = pd.DataFrame()

#Iteration
for j in tqdm(switch):  
    lis = list(j) #Selecting one scenario at a time for iteration
    new_line = pd.DataFrame({'sma20 > sma50': lis[0], 'sma20percent': lis[1], 'momentum': lis[2], 'percentChange': lis[3],\
          'percentOverOpen': lis[4], 'RSI': lis[5], 'volumeRatio': lis[6], 'mktcap': lis[7], 'MACD': lis[8]}, index=[0])
    #Creating new line with the On Off combination for that iteration
    
    #Determing the index of all the OFFs of the particular instance
    for i in range(len(lis)):
        if lis[i] == "OFF":
            indices.append(i)
    
    #Getting the columns names of OFFs of the particular iteration
    for c in indices:
        col = df.columns[c]
        col_name.append(col)
    
    temp = df.drop(columns = col_name) #Drop all the OFF columns
    try:
        df_win = temp[temp['win'] == 1] #Filter the dataset where win is 1
        df_win["count"] = df_win.sum(axis=1) #Create a new column and sum each row
        win_count = df_win["count"].value_counts() #Get the frequency of result at store in the series
        win_filter = win_count.to_frame() #Convert the series to Dataframe
        win_filter = win_filter.reset_index() #Reset index of the dataframe to access both sum number and its frequency
        win_filter = win_filter[win_filter['index'] == 1] #Filter the datframe to get frequency of 1 (win)
        win_filter = win_filter.reset_index(drop = True) #Reset the index so that index of filter value is 0
        win = win_filter['count'].values[0]
    except:
        win = 0
        
    new_line['win'] = win #Add another column and store the win value
    
    #Repeat the process for Update Win, Loss & Update Loss
    try:
        df_upwin = temp[temp['upwin'] == 1]
        df_upwin["count"] = df_upwin.sum(axis=1)
        upwin_count = df_upwin["count"].value_counts()
        upwin_filter = upwin_count.to_frame()
        upwin_filter = upwin_filter.reset_index()
        upwin_filter = upwin_filter[upwin_filter['index'] == 1]
        upwin_filter = upwin_filter.reset_index(drop = True)
        upwin = upwin_filter['count'].values[0]
    except:
        upwin = 0
    
    new_line['upwin'] = upwin

    try:
        df_loss = temp[temp['Loss'] == 1]
        df_loss["count"] = df_loss.sum(axis=1)
        loss_count = df_loss["count"].value_counts()
        loss_filter = loss_count.to_frame()
        loss_filter = loss_filter.reset_index()
        loss_filter = loss_filter[loss_filter['index'] == 1]
        loss_filter = loss_filter.reset_index(drop = True)
        loss = loss_filter['count'].values[0]
    except:
        loss = 0
    
    new_line['Loss'] = loss

    try:
        df_uploss = temp[temp['update loss'] == 1]
        df_uploss["count"] = df_uploss.sum(axis=1)
        uploss_count = df_uploss["count"].value_counts()
        uploss_filter = uploss_count.to_frame()
        uploss_filter = uploss_filter.reset_index()
        uploss_filter = uploss_filter[uploss_filter['index'] == 1]
        uploss_filter = uploss_filter.reset_index(drop = True)
        uploss = uploss_filter['count'].values[0]
    except:
        uploss = 0
    
    new_line['update loss'] = uploss
    
    result = result.append(new_line) #Add the new line in the empty dataframe created earlier
    indices.clear() #Empty the list for next iteration
    col_name.clear() #Empty the list for next iteration

result['total'] = result['win'] + result['upwin'] + result['Loss'] + result['update loss'] #Get the total number of events
result_filter = result[result['total'] >= min_event] #Filter the iterations which has less than threshold events

#Assign weight to each event
result_filter['winweight'] = result_filter['win'] * 2
result_filter['upwinweight'] = result_filter['upwin'] * 1
result_filter['lossweight'] = result_filter['Loss'] * 2.25
result_filter['uplossweight'] = result_filter['update loss'] * 1.5

#Calculate the Win/Loss Ratio
if (result_filter['lossweight'] + result_filter['uplossweight'] == 0).any():
    result_filter['Win/Loss Ratio'] = 0
else:
    result_filter['Win/Loss Ratio'] = (result_filter['winweight'] + result_filter['upwinweight']) / (result_filter['lossweight'] + result_filter['uplossweight'])
                
result_filter = result_filter.drop(columns = ['total', 'winweight', 'upwinweight', 'lossweight', 'uplossweight'])
#Drop Unneccessary columns

result_sort = result_filter.sort_values(by = ['Win/Loss Ratio'], ascending=False) #Sort the data according Win/Loss Ratio
method2 = result_sort.head() #Pick the top 5 iterations
method2.to_csv('static_method2.csv', index = False) #Save the result in CSV
method2 #Display the result in Ipython

,sma20 > sma50,sma20percent,momentum,percentChange,percentOverOpen,RSI,volumeRatio,mktcap,MACD,win,upwin,Loss,update loss,Win/Loss Ratio
0,ON,ON,OFF,ON,OFF,OFF,OFF,OFF,ON,36,0,21,0,1.523810
0,ON,ON,OFF,OFF,OFF,OFF,OFF,OFF,ON,36,0,21,0,1.523810
0,ON,ON,ON,ON,OFF,OFF,OFF,OFF,OFF,44,0,26,0,1.504274
0,ON,ON,ON,OFF,OFF,OFF,OFF,OFF,OFF,44,0,26,0,1.504274
0,ON,ON,ON,ON,ON,OFF,OFF,OFF,OFF,27,0,19,0,1.263158


## **User Input for Static - Method 1**

In [6]:
#User Input give the value low, high and step (IN THAT ORDER ONLY)
lower_bound_sma20 = -5,-3,2
upper_bound_sma20 = -2,-1,1
momentum_threshold = -50,50,25
up_from_day = 5,10,4
up_from_open = 10,14,2
lower_bound_rsi = 50,70,15
upper_bound_rsi = 60,80,15
average_volume_ratio = 0.5,2.5,1.5
market_cap_threshold = 500,5000,2000

sma20_isgreaterthan_sma50 = 'OFF'
sma20percent = 'OFF'
momentum = 'ON'
percent_change = 'OFF'
percent_over_open = 'ON'
rsi = 'ON'
volume_ratio = 'OFF'
mktcap = 'ON'
macd = 'OFF'

min_event = 30

## **Static - Method 1**

In [7]:
switch = [sma20_isgreaterthan_sma50, sma20percent, momentum, percent_change, percent_over_open, rsi, volume_ratio, \
          mktcap, macd] #Get all the swith (on off in a list)

off_index = [i for i, x in enumerate(switch) if x == "OFF"] #Get the indices of OFF in a list
off_column = off_index.copy() #Get the copy of above list. This will be used later

#If the indices contains 0 or 8. Remove it. Since 'sma20 > sma50 check' and 'MACD pass/fail check' doesnt have value
if 0 in off_index:
    off_index.remove(0)    
if 8 in off_index:
    off_index.remove(8)

del_col = [x+1 if x >= 6 else x for x in off_index] #Since rsi lower and upper bound are sharing a switch add 1 to variable coming after

#Since 2 variables are sharing switch. Add another index if the other exist aligns variables indices with switch's (OFF only)
for l in del_col:
    if (l == 1):
        del_col.append(0)
    if (l == 5):
        del_col.append(6)

del_col.sort() #Sort the indices of off (THIS IS IMPORTANT)

threshold_list = [lower_bound_sma20, upper_bound_sma20, momentum_threshold, up_from_day, up_from_open, lower_bound_rsi, \
                 upper_bound_rsi, average_volume_ratio, market_cap_threshold] #Get the low high step of variables in a list

#Remove variables whose switch is off to reduce no of iterations
for ele in sorted(del_col, reverse = True): 
    del threshold_list[ele]

#Expand low, high with step of all variables. Eg: low 500 high 5000 step 2000 will give 500, 2500, 4500.
iteration = []
for s in threshold_list:
    lis = list(s)
    start = [lis[0]]
    low = lis[0]
    while low <= s[1]:
        low += s[2]
        start.append(low)
    start.pop()
    iteration.append(start)

iterate = list(itertools.product(*iteration)) #Create all possible iterations

#Since 'sma20 > sma50 check' and 'MACD pass/fail check' doesnt have variables create its 0, 1 columns
data.loc[(data['20SMA'] > data['50SMA'] ) , 'sma20 > sma50'] = 0
data['sma20 > sma50'] = data['sma20 > sma50'].fillna(1)
data.loc[(data['Macd'] == 'Pass') , 'MACD'] = 0
data['MACD'] = data['MACD'].fillna(1)

data['mrkt cap'] = data['mrkt cap'].fillna(0) #Fill Blank space in market cap with 0
data['Volume Ratio'] = data['Volume Ratio'].fillna(0) #Fill Blank space in volume ratio with 0

result = pd.DataFrame() #Create an empty DataFrame to store result
col_name = [] #Create an empty list

event = ['win', 'upwin', 'Loss', 'update loss'] #Get all the events in a list

#Add 'OFF' in the specified index for all the iterations created earlier
scenarios = []
for j in iterate:  
    lis = list(j)
    
    for k in del_col:
        lis.insert(k, 'OFF')
    scenarios.append(lis)

for lis in tqdm(scenarios): #Loop Starts
    
    #Create columns with default values. Will change when conditions are met
    data['sma20 bound check'] = 1
    data['momentum'] = 1
    data['8% Daily'] = 1
    data['2% UP'] = 1
    data['RSI (60-72)'] = 1
    data['Volume Ration'] = 1
    data['market cap'] = 0
    
    #Change the values created in column created above based on conditions. Will ignore if it is 'OFF'
    try:
        data.loc[(data['20SMA %'] < lis[1]) & (data['20SMA %'] > lis[0]), 'sma20 bound check'] = 0    
    except:
        pass
    try:
        data.loc[(data['Mom'] > lis[2]) , 'momentum'] = 0
    except:
        pass
    try:
        data.loc[(data['%Up'] > lis[3]) , '8% Daily'] = 0
    except:
        pass
    try:
        data.loc[(data['% Open'] > lis[4]) , '2% UP'] = 0
    except:
        pass
    try:
        data.loc[(data['14minRSI'] < lis[6]) & (data['14minRSI'] > lis[5]), 'RSI (60-72)'] = 0
    except:
        pass
    try:
        data.loc[(data['Volume Ratio'] < lis[7]) , 'Volume Ration'] = 0
    except:
        pass
    try:
        data.loc[(data['mrkt cap'] < lis[8]) , 'market cap'] = 1
    except:
        pass

    df = data[['sma20 > sma50','sma20 bound check', 'momentum', '8% Daily', '2% UP', 'RSI (60-72)', 'Volume Ration', \
           'market cap', 'MACD', 'win', 'upwin', 'Loss', 'update loss' ]] #Selecting relevant columns
    
    new_line = pd.DataFrame({'Lower % bound for sma20': lis[0], 'Upper % bound for sma20': lis[1], \
                             'Momentum threshold': lis[2], '% up for Day': lis[3], '% up from open': lis[4], \
          'Lower bound for RSI': lis[5], 'Upper bound for RSI': lis[6], 'Total/Average volume ratio': lis[7], \
                             'Market cap threshold': lis[8]}, index=[0]) #Create new line with the combination of variable for the current iterations
    
    #Get the col names of 'OFF' in the list created earlier
    for c in off_column:
        col = df.columns[c]
        col_name.append(col)

    temp = df.drop(columns = col_name) #Drop all the OFF columns
    
    col_name.clear() #Clear the list 

    for e in event: #Loop to get the number of all events
        try:
            df_event = temp[temp[e] == 1] #Filter the dataset where event is 1
            df_event["count"] = df_event.sum(axis=1) #Create a new column and sum each row
            event_count = df_event["count"].value_counts() #Get the frequency of result at store in the series
            event_filter = event_count.to_frame() #Convert the series to Dataframe
            event_filter = event_filter.reset_index() #Reset index of the dataframe to access both sum number and its frequency
            event_filter = event_filter[event_filter['index'] == 1] #Filter the datframe to get frequency of 1 (event)
            event_filter = event_filter.reset_index(drop = True) #Reset the index so that index of filter value is 0
            no_of_events = event_filter['count'].values[0]
        except:
            no_of_events = 0              

        new_line[e] = no_of_events #Add another column and store the event value

    result = result.append(new_line) #Add the new line in the empty dataframe created earlier
    
    data = data.drop(['sma20 bound check', 'momentum','8% Daily', '2% UP', 'RSI (60-72)', 'Volume Ration', 'market cap'],\
                 axis = 1) #Delete the columns created in the start of loop.

result['total'] = result['win'] + result['upwin'] + result['Loss'] + result['update loss'] #Get the total number of events
result_filter = result[result['total'] >= min_event] #Filter the iterations which has less than threshold events

#Assign weight to each event
result_filter['winweight'] = result_filter['win'] * 2
result_filter['upwinweight'] = result_filter['upwin'] * 1
result_filter['lossweight'] = result_filter['Loss'] * 2.25
result_filter['uplossweight'] = result_filter['update loss'] * 1.5

#Calculate the Win/Loss Ratio
if (result_filter['lossweight'] + result_filter['uplossweight'] == 0).any():
    result_filter['Win/Loss Ratio'] = 0
else:
    result_filter['Win/Loss Ratio'] = (result_filter['winweight'] + result_filter['upwinweight']) / (result_filter['lossweight'] + result_filter['uplossweight'])
                
result_filter = result_filter.drop(columns = ['total', 'winweight', 'upwinweight', 'lossweight', 'uplossweight'])
#Drop Unneccessary columns

result_sort = result_filter.sort_values(by = ['Win/Loss Ratio'], ascending=False) #Sort the data according Win/Loss Ratio
method1 = result_sort.head() #Pick the top 5 iterations
method1.to_csv('static_method1.csv', index = False) #Save the result in CSV
method1 #Display the result

,Lower % bound for sma20,Upper % bound for sma20,Momentum threshold,% up for Day,% up from open,Lower bound for RSI,Upper bound for RSI,Total/Average volume ratio,Market cap threshold,win,upwin,Loss,update loss,Win/Loss Ratio
0,OFF,OFF,0,OFF,14,65,75,OFF,500,24,0,17,0,1.254902
0,OFF,OFF,-25,OFF,14,65,75,OFF,2500,18,0,13,0,1.230769
0,OFF,OFF,25,OFF,14,50,75,OFF,2500,46,0,32,3,1.202614
0,OFF,OFF,25,OFF,14,50,60,OFF,2500,22,0,15,3,1.150327
0,OFF,OFF,50,OFF,14,50,75,OFF,2500,44,0,32,3,1.150327


##  **User Input for Dynamic Method**

In [5]:
#User Input give the value low, high and step (IN THAT ORDER ONLY)
lower_bound_sma20 = -5,-3,2
upper_bound_sma20 = -2,-1,1
momentum_threshold = -50,50,25
up_from_day = 5,10,4
up_from_open = 10,14,2
lower_bound_rsi = 50,70,15
upper_bound_rsi = 60,80,15
average_volume_ratio = 0.5,2.5,1.5
market_cap_threshold = 500,5000,2000
fit_equation = 2.4,2.7,0.3 #New Line

sma20_isgreaterthan_sma50 = 'ON', 'OFF'
sma20percent = 'ON', 'OFF'
momentum = 'ON', 'OFF'
percent_change = 'ON', 'OFF'
percent_over_open = 'ON', 'OFF'
rsi = 'ON',
volume_ratio = 'ON', 'OFF'
mktcap = 'ON',
macd = 'ON', 'OFF'
fit_eq = 'ON', 'OFF' #New Line

min_event = 30

## **Dynamic**

In [ ]:
data['mrkt cap'] = data['mrkt cap'].fillna(0) #Fill Blank space in market cap with 0 
data['Volume Ratio'] = data['Volume Ratio'].fillna(0) #Fill Blank space in volume ratio with 0
data['Fit EQ'] = data['Fit EQ'].fillna(0) #Fill Blank space in Fit EQ with 0

switch = list(itertools.product(sma20_isgreaterthan_sma50, sma20percent, momentum, percent_change, percent_over_open, rsi,\
                       volume_ratio, mktcap, fit_eq, macd)) #Get all the permutation of 'ON' and 'OFF' FITEQ ADDED

result = pd.DataFrame() #Create an empty DataFrame to store result
col_name = [] #Create an empty list

event = ['win', 'upwin', 'Loss', 'update loss'] #Get all the events in a list

for j in tqdm(switch): #Iterate
    switch_lis = list(j) #Get the current permutation in a list

    #Assign default value to column will change if the condition is met
    data['sma20 > sma50'] = 1 
    data['MACD'] = 1
    try:
        data.loc[(data['20SMA'] > data['50SMA'] ) , 'sma20 > sma50'] = 0
    except:
        pass
    try:
        data.loc[(data['Macd'] == 'Pass') , 'MACD'] = 0
    except:
        pass

    off_index = [i for i, x in enumerate(switch_lis) if x == "OFF"] #Get the indices of OFF in a list
    off_column = off_index.copy() #Get the copy of above list. This will be used later
    #If the indices contains 0 or 9 . Remove it. Since 'sma20 > sma50 check' and 'MACD pass/fail check' doesnt have value
    if 0 in off_index:
        off_index.remove(0)    
    if 9 in off_index:
        off_index.remove(9)

    del_col = [x+1 if x >= 6 else x for x in off_index] #Since rsi lower and upper bound are sharing a switch add 1 to variable coming after

    #Since 2 variables are sharing switch. Add another index if the other exist aligns variables indices with switch's (OFF only)
    for l in del_col:
        if (l == 1):
            del_col.append(0)
        if (l == 5):
            del_col.append(6)

    del_col.sort() #Sort the indices of off (THIS IS IMPORTANT)

    threshold_list = [lower_bound_sma20, upper_bound_sma20, momentum_threshold, up_from_day, up_from_open, lower_bound_rsi, \
                     upper_bound_rsi, average_volume_ratio, market_cap_threshold, fit_equation] #Get the low high step of variables in a list FITEQ ADDED
    
    #Remove variables whose switch is off to reduce no of iterations
    for ele in sorted(del_col, reverse = True): 
        del threshold_list[ele]

    #Expand low, high with step of all variables. Eg: low 500 high 5000 step 2000 will give 500, 2500, 4500
    iteration = []
    for s in threshold_list:
        lis = list(s)
        start = [lis[0]]
        low = lis[0]
        while low <= s[1]:
            low += s[2]
            start.append(low)
        start.pop()
        iteration.append(start)
    
    iterate = list(itertools.product(*iteration)) #Create all possible iterations
    
    #Add 'OFF' in the specified index for all the iterations created in line 66 line 86 to 92
    scenarios = []
    for j in iterate:  
        lis = list(j)
        for k in del_col:
            lis.insert(k, 'OFF')
        scenarios.append(lis)
        
    for lis in tqdm(scenarios): #Variable Loop
        #Create columns with default values. Will change when conditions are met
        data['sma20 bound check'] = 1
        data['momentum'] = 1
        data['8% Daily'] = 1
        data['2% UP'] = 1
        data['RSI (60-72)'] = 1
        data['Volume Ration'] = 1
        data['market cap'] = 0
        data['FitEQ'] = 0 #New Line Added

        #Change the values created in column created above based on conditions. Will ignore if it is 'OFF'
        try:
            data.loc[(data['20SMA %'] < lis[1]) & (data['20SMA %'] > lis[0]), 'sma20 bound check'] = 0    
        except:
            pass
        try:
            data.loc[(data['Mom'] > lis[2]) , 'momentum'] = 0
        except:
            pass
        try:
            data.loc[(data['%Up'] > lis[3]) , '8% Daily'] = 0
        except:
            pass
        try:
            data.loc[(data['% Open'] > lis[4]) , '2% UP'] = 0
        except:
            pass
        try:
            data.loc[(data['14minRSI'] < lis[6]) & (data['14minRSI'] > lis[5]), 'RSI (60-72)'] = 0
        except:
            pass
        try:
            data.loc[(data['Volume Ratio'] < lis[7]) , 'Volume Ration'] = 0
        except:
            pass
        try:
            data.loc[(data['mrkt cap'] < lis[8]) , 'market cap'] = 1
        except:
            pass
        try: #New Line Added
            data.loc[(data['Fit EQ'] < lis[9]) , 'FitEQ'] = 1
        except:
            pass

        df = data[['sma20 > sma50','sma20 bound check', 'momentum', '8% Daily', '2% UP', 'RSI (60-72)', 'Volume Ration', \
                   'market cap', 'MACD', 'win', 'FitEQ', 'upwin', 'Loss', 'update loss']] #Selecting relevant columns FITEQ ADDED

        new_line = pd.DataFrame({'sma20 > sma50 check': switch_lis[0], 'Lower % bound for sma20': lis[0], \
                'Upper % bound for sma20': lis[1], 'Momentum threshold': lis[2], '% up for Day': lis[3], '% up from open': lis[4], \
                'Lower bound for RSI': lis[5], 'Upper bound for RSI': lis[6], 'Total/Average volume ratio': lis[7], \
              'Market cap threshold': lis[8], 'Fit equation threshold': lis[9], 'MACD': switch_lis[8]}, index=[0]) #Create new line with the combination of variable for the current iterations FITEQ ADDED

        #Get the col names of 'OFF' in the list created earlier
        for c in off_column:
            col = df.columns[c]
            col_name.append(col)

        temp = df.drop(columns = col_name) #Drop all the OFF columns

        col_name.clear() #Clear the list 

        for e in event: #Loop to get the number of all events
            try:
                df_event = temp[temp[e] == 1] #Filter the dataset where event is 1
                df_event["count"] = df_event.sum(axis=1) #Create a new column and sum each row
                event_count = df_event["count"].value_counts() #Get the frequency of result at store in the series
                event_filter = event_count.to_frame() #Convert the series to Dataframe
                event_filter = event_filter.reset_index() #Reset index of the dataframe to access both sum number and its frequency
                event_filter = event_filter[event_filter['index'] == 1] #Filter the datframe to get frequency of 1 (event)
                event_filter = event_filter.reset_index(drop = True) #Reset the index so that index of filter value is 0
                no_of_events = event_filter['count'].values[0]
            except:
                no_of_events = 0

            new_line[e] = no_of_events #Add another column and store the event value

        result = result.append(new_line) #Add the new line in the empty dataframe created earlier

        data = data.drop(['sma20 bound check', 'momentum','8% Daily', '2% UP', 'RSI (60-72)', 'Volume Ration', 'market cap'],\
                     axis = 1) #Delete the columns created in the start of loop.
result['total'] = result['win'] + result['upwin'] + result['Loss'] + result['update loss'] #Get the total number of events
result_filter = result[result['total'] >= min_event] #Filter the iterations which has less than threshold events

#Assign weight to each event
result_filter['winweight'] = result_filter['win'] * 2
result_filter['upwinweight'] = result_filter['upwin'] * 1
result_filter['lossweight'] = result_filter['Loss'] * 2.25
result_filter['uplossweight'] = result_filter['update loss'] * 1.5

#Calculate the Win/Loss Ratio
if (result_filter['lossweight'] + result_filter['uplossweight'] == 0).any():
    result_filter['Win/Loss Ratio'] = 0
else:
    result_filter['Win/Loss Ratio'] = (result_filter['winweight'] + result_filter['upwinweight']) / (result_filter['lossweight'] + result_filter['uplossweight'])
                
result_filter = result_filter.drop(columns = ['total', 'winweight', 'upwinweight', 'lossweight', 'uplossweight'])
#Drop Unneccessary columns

result_sort = result_filter.sort_values(by = ['Win/Loss Ratio'], ascending=False) #Sort the data according Win/Loss Ratio
dynamic = result_sort.head() #Pick the top 5 iterations
dynamic.to_csv('dynamic.csv', index = False) #Save the result in CSV
dynamic #Display the result

